In [96]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Model, save_model, load_model
from keras.layers import Input, LSTM, Dense, Flatten
from keras.callbacks import EarlyStopping

## 전체데이터(104만개) 불러오기

In [127]:
train_df = pd.read_csv('train_df.csv', index_col=0)

print('전체 데이터 :', train_df.shape)
train_df.head()

전체 데이터 : (1040000, 2)


C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,중국어,한국어
0,"希望能迈开宝贵的脚步,为孩子们的闪耀梦想加油。",\t부디 귀한 걸음 해주셔서 아이들의 반짝이는 꿈을 응원해 주시면 감사하겠습니다 .\n
1,"根据童子军年度运营计划,将参加由韩国童子军首尔北部联盟江北地区联合会主办的夏令营。",\t스카우트 연간운영계획에 따라 한국스카우트 서울북부연맹 강북지구연합회에서 주최하는...
2,"通过以制造者教育为基础的前进教育,培养学生可以准备幸福未来的积极的前进价值观,举行了乌克丽丽...","\t메이커 교육에 기반을 둔 진로교육을 통해 , 학생들의 행복한 미래를 준비할 수 ..."
3,"总是对周围的各种事物或现象感兴趣,用新的眼睛看待。",\t주변의 여러 사물이나 현상에 늘 관심을 가지고 새로운 눈으로 바라봅니다 .\n
4,这时脖子后面或耳朵后面的头发也一定要梳。,\t이때 목 뒤나 귀 뒤의 머리칼도 반드시 빗질해야 한다 .\n


In [108]:
max([len(li) for li in train_df['중국어']]) , max([len(li) for li in train_df['한국어']]) 

(259, 373)

## 토큰화

In [131]:
def tokenize(train_df):
    #글자단위 토큰화
    ch_vocab, ko_vocab = set(), set()

    for line in train_df['중국어']:
        for c in line:
            ch_vocab.add(c)

    for line in train_df['한국어']:
        for c in line:
            ko_vocab.add(c)
            
    ch_vocab_size = len(ch_vocab) + 1  #94
    ko_vocab_size = len(ko_vocab) + 1  #4837
    
    #set -> list(데이터 변경 용이한 자료구조로 변환)
    ch_vocab = sorted(list(ch_vocab))
    ko_vocab = sorted(list(ko_vocab))
    
    ch_to_index = dict([(c, i+1) for i, c in enumerate(ch_vocab)])
    ko_to_index = dict([(c, i+1) for i, c in enumerate(ko_vocab)])
    
    #중국어 문장 인코딩
    encoder_input = []
    for li in train_df['중국어']:
        t = []
        for c in li:
            t.append(ch_to_index[c])
        encoder_input.append(t)
        
    #한국어 문장 인코딩
    decoder_input = []
    for li in train_df['한국어']:
        t = []
        for c in li:
            t.append(ko_to_index[c])
        decoder_input.append(t)   
        
    #번역되어 나올 한국어 문장 인코딩에서 '\t' 제거
    decoder_ko = []
    for li in train_df['한국어']:
        t = []
        i = 0
        for c in li:
            if i > 0:
                t.append(ko_to_index[c])
            i += 1
        decoder_ko.append(t)    
     
    #패딩
    max_len_ch = 1689
    max_len_ko = 373
    
    #문장 -> int -> padding
    encoder_input = pad_sequences(encoder_input, maxlen=max_len_ch, padding='post')
    decoder_input = pad_sequences(decoder_input, maxlen=max_len_ko, padding='post')
    decoder_ko = pad_sequences(decoder_ko, maxlen=max_len_ko, padding='post') 
    
    #문장들을 3차원 배열로 변환 : (encoder_input, decoder_input, decoder_target)
    #encoder_input은 (문장 개수, 문장 최대 길이, 문자 종류 수) 형태의 3차원 배열로 중국어 문장의 one-hot 형식 벡터 데이터
    #decoder_input은 (문장 개수, 문장 최대 길이, 문자 종류 수) 형태의 3차원 배열로 한국어 문장의 one-hot 형식 벡터 데이터
    #decoder_ko은 decoder_input과 같지만, 하나의 time step만큼 offset, 즉, decoder_target[:, t, :] = decoder_input[:, t+1, :]
    encoder_input = np_utils.to_categorical(encoder_input)
    decoder_input = np_utils.to_categorical(decoder_input)
    decoder_ko = np_utils.to_categorical(decoder_ko)
    
    return encoder_input, decoder_input, decoder_ko, ch_vocab_size, ko_vocab_size, index_to_ch, index_to_ko

#### 데이터가 너무 많으면 한번에 토큰화할 수 없기 때문에,
#### 데이터를 4000개씩 나누어 토큰화 > 모델학습 > 저장 > 전이학습 반복

In [140]:
df = train_df[:4000]
    
encoder_input, decoder_input, decoder_ko, ch_vocab_size, ko_vocab_size, index_to_ch, index_to_ko = tokenize(df)

#중국어 인코딩
tmp_dict = dict((i,c ) for c , i in index_to_ch.items()) 

for i in tmp_dict:
    try:
        tmp_dict[i] = tmp_dict[i].encode('EUC_CN')
    except:
        pass
    
index_to_ch = dict((i,c ) for c , i in tmp_dict.items()) 

In [139]:
# 트레이닝 시 이전 상태의 실제값을 현재상태의 디코더 입력으로 해야함(예측값으로 하면 안됨)
encoder_inputs = Input(shape=(None, ch_vocab_size), name='encoder_input')
decoder_inputs = Input(shape=(None, ko_vocab_size ), name='decoder_input')

# 인코더 LSTM 셀
encoderLSTM = LSTM(units=256, return_state=True, name='encoderLSTM')    #return_state :인코더의 마지막 상태 정보를 디코더의 입력 상태 정보로 전달
decoderLSTM = LSTM(units=256, return_sequences=True, return_state=True, name='decoderLSTM')

# 인코더 LSTM셀의 입력 정의
encoder_outputs, stateH, stateC = encoderLSTM(encoder_inputs) # _, 히든상태(위), 셀상태(오른쪽)
encoder_state = [stateH, stateC] # 컨텍스트 벡터

decoder_output, _, _ = decoderLSTM(decoder_inputs, initial_state=encoder_state)
decoder_softmax = Dense(ko_vocab_size, activation="softmax")
decoder_output = decoder_softmax(decoder_output)

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_output)

model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, None, 1048)  0           []                               
                                ]                                                                 
                                                                                                  
 decoder_input (InputLayer)     [(None, None, 598)]  0           []                               
                                                                                                  
 encoderLSTM (LSTM)             [(None, 256),        1336320     ['encoder_input[0][0]']          
                                 (None, 256),                                                     
                                 (None, 256)]                                              

In [6]:
model.compile(optimizer="adam", loss="categorical_crossentropy")

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.fit(x=[encoder_input,decoder_input], y=decoder_ko, batch_size=64, epochs=50, callbacks=early_stopping)
save_model(model, 'ch_to_ko.h5', overwrite=True)

Epoch 1/50
40/40 [==============================] - 169s 4s/step - loss: 3.1127
Epoch 2/50
40/40 [==============================] - 189s 5s/step - loss: 1.2821
Epoch 3/50
40/40 [==============================] - 204s 5s/step - loss: 1.1769
Epoch 4/50
40/40 [==============================] - 195s 5s/step - loss: 1.2813
Epoch 5/50
40/40 [==============================] - 189s 5s/step - loss: 1.2869
Epoch 6/50
40/40 [==============================] - 189s 5s/step - loss: 1.8179
Epoch 7/50
40/40 [==============================] - 191s 5s/step - loss: 1.0270
Epoch 8/50
40/40 [==============================] - 191s 5s/step - loss: 2.0911
Epoch 9/50
40/40 [==============================] - 190s 5s/step - loss: 1.1776
Epoch 10/50
40/40 [==============================] - 193s 5s/step - loss: 0.9156
Epoch 11/50
40/40 [==============================] - 189s 5s/step - loss: 0.8439
Epoch 12/50
40/40 [==============================] - 190s 5s/step - loss: 0.8238
Epoch 13/50
40/40 [==================

In [ ]:
for i in range(1,len(train_df) // 2500):
    df = train_df[i*(2500):(i+1)*2500]
    
    encoder_input, decoder_input, decoder_ko, ch_vocab_size, ko_vocab_size = tokenize(df)

    model = load_model('ch_to_ko.h5')

    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    model.fit(x=[encoder_input, decoder_input], y=decoder_ko, batch_size=64, epochs=3, callbacks=early_stopping)
    save_model(model, 'ch_to_ko.h5', overwrite=True)

Epoch 1/30


ValueError: in user code:

    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\keras\engi

In [50]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_state)

ch_to_index = dict((i,c ) for c , i in index_to_ch.items()) 
ko_to_index = dict((i,c ) for c , i in index_to_ko.items()) 

In [21]:
encoder_model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, None, 2088)]      0         
                                                                 
 encoderLSTM (LSTM)          [(None, 256),             2401280   
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 2,401,280
Trainable params: 2,401,280
Non-trainable params: 0
_________________________________________________________________


## 기본 LSTM 기반의 seq2seq 모델을 이용해 decoder_ko 예측

In [22]:
# 디코더
decoder_state_input_hidden = Input(shape=(256,))
decoder_state_input_cell = Input(shape=(256,))
decoder_state_input = [decoder_state_input_hidden, decoder_state_input_cell]

decoder_output, state_hidden, state_cell = decoderLSTM(decoder_inputs, initial_state = decoder_state_input)
decoder_state = [state_hidden, state_cell]
decoder_outputs = decoder_softmax(decoder_output)

decoder_model = Model(inputs=[decoder_inputs]+decoder_state_input, outputs=[decoder_output]+decoder_state)

In [23]:
decoder_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input (InputLayer)     [(None, None, 1035)  0           []                               
                                ]                                                                 
                                                                                                  
 input_7 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 decoderLSTM (LSTM)             [(None, None, 256),  1323008     ['decoder_input[0][0]',    

In [142]:
def decode_seq(input_seq): 
    
    state_value = encoder_model.predict(input_seq)
    print('encoder_model의 예상 state_value :',np.shape(state_value))
    
    target_seq = np.zeros((1,1,ko_vocab_size))   #(1, 1, 1134)
    target_seq[0,0,ko_to_index['\t']] = 1      # 원핫인코딩
    
    stop = False
    decoded_sent=""
    while not stop: # "\n"문자를 만날때까지 반복
        
        output, h, c = decoder_model.predict([target_seq]+state_value)
        # 예측값을 한국어 문자로 변환
        token_index = np.argmax(output[0,-1,:]) 
        pred_char = index_to_ko[token_index]
        
        # 현시점 예측문자가 예측문장에 추가
        decoded_sent += pred_char
        
        if (pred_char == "\n" or len(decoded_sent) > 373):
            stop = True
            
        # 현시점 예측결과가 다음 시점에 입력으로 
        target_seq = np.zeros((1,1,ko_vocab_size))
        target_seq[0,0,token_index] = 1
        
        # 현시점 상태를 다음 시점 상태로 사용
        state_value = [h,c]
    
    return decoded_sent # 번역결과

In [143]:
for seq_index in [1,50,100,200,300]:
    
    input_seq = encoder_input[seq_index:seq_index+1]    # (1, 117, 2326)
    decoded_seq = decode_seq(input_seq)
    
    print("입력문장:", train_df['중국어'][seq_index])
    print("정답:", train_df['한국어'][seq_index][1:len(train_df['한국어'][seq_index])-1])   # "\t", "\n" 제거
    print("번역기:", decoded_seq[:len(decoded_seq)-1])
    print("\n")

ValueError: in user code:

    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\yeonok\anaconda3\envs\deeplearning\lib\site-packages\k

## 모델이 잘 작동하는지 확인하기 위해 일부 문장 디코딩
    -encoder_input을 샘플링해 decoder_target으로 변환해본다.